In [ ]:
!pip install kaggle # install kaggle

## For Downloading the Data from kaggle 
Follow below steps:
1. First create new api token from your kaggle profile. Kaggle.json file will be start downloading. 
2. Upload this file on colab.
3. Create /.kaggle directory then copy the kaggle.json file and paste it in that particular folder.
4. Than go to the data which you want to download from there copy api command.
5. Than paste it, so the data will start downloading...



In [ ]:
# for downloading the data from kaggle upload the file we have already downloaded 
from google.colab import files 
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tariqnawaz","key":"010fc9886d25d732e37c09f9e03861f5"}'}

In [ ]:
!mkdir -p ~/.kaggle # make directory 
!cp kaggle.json ~/.kaggle/ # copy kaggle.json to /.kaggle/ directory
!chmod 600 ~/.kaggle/kaggle.json # give persion

In [ ]:
# this code 
!kaggle competitions download -c fake-news 

  0% 0.00/40.6k [00:00<?, ?B/s]
100% 40.6k/40.6k [00:00<00:00, 17.8MB/s]
 76% 28.0M/37.0M [00:00<00:00, 53.8MB/s]
100% 37.0M/37.0M [00:00<00:00, 93.6MB/s]
 96% 9.00M/9.42M [00:00<00:00, 90.3MB/s]
100% 9.42M/9.42M [00:00<00:00, 86.8MB/s]


In [4]:
from zipfile import ZipFile
file_name = "train.csv.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("Done")

Done


# Fake News Classifier Using LSTM
Dataset: https://www.kaggle.com/c/fake-news/data#

In [5]:
import pandas as pd


In [6]:
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
# Drop nan values
df = df.dropna()

In [8]:
# Get the Independent features
X = df.drop('label', axis = 1)

In [9]:
## Get the Dependent features
y = df['label']

In [10]:

X.shape

(18285, 4)

In [11]:
y.shape

(18285,)

In [12]:
import tensorflow as tf
tf.__version__

'2.7.0'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [14]:
### vocabulary size
voc_size = 5000

In [15]:
# OneHot Representation
messages= X.copy()

In [16]:
messages['text'][1]

'Ever get the feeling your life circles the roundabout rather than heads in a straight line toward the intended destination? [Hillary Clinton remains the big woman on campus in leafy, liberal Wellesley, Massachusetts. Everywhere else votes her most likely to don her inauguration dress for the remainder of her days the way Miss Havisham forever wore that wedding dress.  Speaking of Great Expectations, Hillary Rodham overflowed with them 48 years ago when she first addressed a Wellesley graduating class. The president of the college informed those gathered in 1969 that the students needed “no debate so far as I could ascertain as to who their spokesman was to be” (kind of the like the Democratic primaries in 2016 minus the   terms unknown then even at a Seven Sisters school). “I am very glad that Miss Adams made it clear that what I am speaking for today is all of us —  the 400 of us,” Miss Rodham told her classmates. After appointing herself Edger Bergen to the Charlie McCarthys and Mor

In [17]:
messages.reset_index(inplace=True)

In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [174]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i]) # look at this here we just take the News title and base on the we will train the model
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [175]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[1322, 1868, 458, 1221, 2007, 2109, 4791, 4809, 187, 138],
 [3180, 2257, 3502, 409, 2147, 1622, 859],
 [3793, 2956, 2580, 1971],
 [4686, 3985, 2314, 4444, 1521, 4712],
 [4012, 2147, 913, 993, 1197, 222, 2147, 4650, 3363, 210],
 [3605,
  3458,
  4785,
  4183,
  2849,
  3389,
  781,
  1855,
  4882,
  879,
  3450,
  2179,
  3175,
  1503,
  859],
 [188, 1961, 354, 2925, 972, 3101, 3833, 1317, 1534, 3025, 3557],
 [166, 729, 960, 2847, 4846, 2797, 3389, 2165, 1534, 3025, 3557],
 [3330, 156, 2893, 4800, 1273, 1427, 1294, 2893, 3389, 4001],
 [4586, 4073, 3056, 2602, 1605, 3303, 216, 3593],
 [3721, 2057, 2581, 194, 517, 1168, 1500, 512, 3397, 398, 2922],
 [4444, 2465, 2007, 1427, 3389, 4846],
 [121, 1416, 4150, 3159, 2421, 636, 2900, 2500, 201],
 [2401, 479, 3213, 70, 3931, 609, 482, 1534, 3025, 3557],
 [4750, 1616, 3124, 1541, 3590, 1534, 3025, 3557],
 [4725, 931, 1349, 3265, 3894, 3752, 4125, 4229, 877, 3603],
 [3813, 2325, 2257],
 [4876, 1590, 157, 1897, 3389, 2534, 2591, 859],
 [1033, 2182

In [176]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4809  187  138]
 [   0    0    0 ... 2147 1622  859]
 [   0    0    0 ... 2956 2580 1971]
 ...
 [   0    0    0 ... 1534 3025 3557]
 [   0    0    0 ... 1721 3055 3307]
 [   0    0    0 ...  971 2108 2079]]


In [177]:
embedded_docs[0]


array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1322,
       1868,  458, 1221, 2007, 2109, 4791, 4809,  187,  138], dtype=int32)

In [178]:

## Creating model
embedding_vector_features=80
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(128))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
print(model.summary())

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 20, 80)            400000    
                                                                 
 lstm_19 (LSTM)              (None, 128)               107008    
                                                                 
 dense_14 (Dense)            (None, 1)                 129       
                                                                 
Total params: 507,137
Trainable params: 507,137
Non-trainable params: 0
_________________________________________________________________
None


In [179]:
len(embedded_docs),y.shape

(18285, (18285,))

In [180]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [181]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

#Model Training

In [182]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [183]:

### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 13s 57ms/step - loss: 0.2815 - accuracy: 0.8767 - val_loss: 0.1967 - val_accuracy: 0.9171
Epoch 2/10
192/192 [==============================] - 10s 53ms/step - loss: 0.1526 - accuracy: 0.9399 - val_loss: 0.2074 - val_accuracy: 0.9201
Epoch 3/10
192/192 [==============================] - 10s 54ms/step - loss: 0.1247 - accuracy: 0.9514 - val_loss: 0.2469 - val_accuracy: 0.9168
Epoch 4/10
192/192 [==============================] - 10s 53ms/step - loss: 0.1060 - accuracy: 0.9602 - val_loss: 0.2030 - val_accuracy: 0.9160
Epoch 5/10
192/192 [==============================] - 10s 53ms/step - loss: 0.0899 - accuracy: 0.9668 - val_loss: 0.2409 - val_accuracy: 0.9228
Epoch 6/10
192/192 [==============================] - 10s 52ms/step - loss: 0.0755 - accuracy: 0.9724 - val_loss: 0.2517 - val_accuracy: 0.9188
Epoch 7/10
192/192 [==============================] - 10s 52ms/step - loss: 0.0635 - accuracy: 0.9771 - val_loss: 0.2651 - val_accuracy:

In [107]:
model.save('FakeNewClassifier.hdf5')

Now again create model and train with some validation data take from train data rather than validate on test data. we will take prediction from test data

In [156]:

### Finally Training
model.fit(X_train,y_train,validation_split=0.1,epochs=10,batch_size=64)

Epoch 1/10
173/173 [==============================] - 13s 61ms/step - loss: 0.2895 - accuracy: 0.8745 - val_loss: 0.2008 - val_accuracy: 0.9224
Epoch 2/10
173/173 [==============================] - 10s 60ms/step - loss: 0.1522 - accuracy: 0.9410 - val_loss: 0.1828 - val_accuracy: 0.9290
Epoch 3/10
173/173 [==============================] - 10s 61ms/step - loss: 0.1249 - accuracy: 0.9517 - val_loss: 0.1957 - val_accuracy: 0.9290
Epoch 4/10
173/173 [==============================] - 10s 57ms/step - loss: 0.1049 - accuracy: 0.9603 - val_loss: 0.2014 - val_accuracy: 0.9322
Epoch 5/10
173/173 [==============================] - 10s 60ms/step - loss: 0.0917 - accuracy: 0.9667 - val_loss: 0.2024 - val_accuracy: 0.9298
Epoch 6/10
173/173 [==============================] - 11s 61ms/step - loss: 0.0783 - accuracy: 0.9719 - val_loss: 0.1766 - val_accuracy: 0.9290
Epoch 7/10
173/173 [==============================] - 11s 62ms/step - loss: 0.0637 - accuracy: 0.9771 - val_loss: 0.1952 - val_accuracy:

so Now Validation Accuracy is some way around 93%

In [157]:
preds = model.predict(X_test)

In [161]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
for i in range(len(preds)):
  if preds[i] > 0.5:
    preds[i] = 1
  else:
    preds[i] = 0

report = classification_report(y_test, preds)


In [163]:
print(report)

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3419
           1       0.90      0.89      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.90      0.91      6035
weighted avg       0.91      0.91      0.91      6035

